# 4.3 - Real Time Location Systems
In this lecture you will get exposed to Python programming and location data. You will learn how to process the raw data output into tangible knowledge that can be used to analyze trajectories of construction assets.
You will encounter several codeblocks with ***TO DO*** written in them. This is where you will have to follow the instructions and fill out the missing code before running the cell.


In [1]:
# Copyright 2022 AU Digital Construction. All Rights Reserved.
# 
# Use or modification of this code outside of the course should reference:
# Emil L. Jacobsen, Karsten W. Johansen, and Christos Chronopoulos 
# Lecture Notes Digital Construction 2022

## 4.3.1 Install dependencies
For this exercise. Some of them will be repeating dependencies for all exercises, while other might only be used in certain examples. For this lecture, we will install the following

In lecture 03.01 we saw how to install the dependencies that might be missing.

Python will trow an error similar to *No module named 'package name'* if the package is not installed



In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from tqdm import tqdm
import traja
%matplotlib qt

## 4.3.2 Load the data
The data is in a csv file for this sort of data we are going to use pandas, which is a library for tourning csv data into a dataframe that we can work with.
Pandas documentation: [Read function](https://pandas.pydata.org/docs/user_guide/io.html#io)

The data that will be the subject for this lecture is RTLS data, more specifically UWB data recorded with the system that we will use or have used on the Demo day. 
The data is written as comma seperated data and the collomns correspond to:

[Data-Header, tag-ID, X, Y, Z,battery, timestamp, unit, DQI, GDOP, Locate-Details, LF]


In [3]:
df = pd.read_csv("data.csv", names=["data_header","tag_id","x","y","z","battery","time","unit","dqi","gdop","locate_details"], 
                                                                                                    converters = {'tag_ID': str})
df.head()

,data_header,tag_id,x,y,z,battery,time,unit,dqi,gdop,locate_details
0,T,0025CB03,12.14,12.50,1.52,12,1.618572e+09,1,0.11,G0.53,S01-S02-S04-S05
1,T,0025CB03,12.01,12.63,1.52,12,1.618572e+09,1,0.29,G0.53,S01-S02-S04-S05
2,T,0025CB03,12.06,12.67,1.52,12,1.618572e+09,1,1.01,G0.53,S01-S02-S04-S05
3,T,0025CB03,12.04,12.64,1.52,12,1.618572e+09,1,0.30,G0.53,S01-S02-S04-S05
4,T,0025CB03,12.01,12.62,1.52,12,1.618572e+09,1,0.29,G0.53,S04-S05-S01-S02


That is a lot of information... 
For our analysis we only need id, position(x,y), and time , therefore we drop the rest for now. Nevertheless the remainding data could be used in future analysis.


In [4]:
df = df[['tag_id','x','y','time']]
df


,tag_id,x,y,time
0,0025CB03,12.14,12.50,1.618572e+09
1,0025CB03,12.01,12.63,1.618572e+09
2,0025CB03,12.06,12.67,1.618572e+09
3,0025CB03,12.04,12.64,1.618572e+09
4,0025CB03,12.01,12.62,1.618572e+09
...,...,...,...,...
4690,0025C9EB,7.86,2.75,1.618572e+09
4691,0025CAF6,2.54,0.30,1.618572e+09
4692,0025CB03,11.93,12.45,1.618572e+09
4693,0025CB05,1.51,11.68,1.618572e+09


## 4.3.3 Basic introductary analysis
Now that the data has been loaded into a pandas dataframe, it may be interesting to figure out how many tags that are present in the data
but are there any smart ways to do so ? 

### 4.3.3.1 Unique id in data
It would be interesting to find the number of unique values in the "tag_id" coloumn ?

This is performed in the code below

In [5]:
unique_tags = df.tag_id.unique()
number_of_unique_tags = len(unique_tags)
print(f"There exists {number_of_unique_tags} in the tag_ID column and their IDs are: \n {unique_tags}")


There exists 5 in the tag_ID column and their IDs are: 
 ['0025CB03' '0025C9EB' '0025C9E6' '0025CAF6' '0025CB05']


### 4.3.3.2 Data duration

Does the time format seem wierd? [epoch time](https://www.epochconverter.com/)

The below code fixed this epoch time issue

In [6]:
df.time = pd.to_datetime(df.time.values.astype(float),unit='s')
df

,tag_id,x,y,time
0,0025CB03,12.14,12.50,2021-04-16 11:16:20.170000128
1,0025CB03,12.01,12.63,2021-04-16 11:16:20.306999808
2,0025CB03,12.06,12.67,2021-04-16 11:16:20.441999872
3,0025CB03,12.04,12.64,2021-04-16 11:16:20.579000064
4,0025CB03,12.01,12.62,2021-04-16 11:16:20.716999936
...,...,...,...,...
4690,0025C9EB,7.86,2.75,2021-04-16 11:19:15.657000192
4691,0025CAF6,2.54,0.30,2021-04-16 11:19:15.726000128
4692,0025CB03,11.93,12.45,2021-04-16 11:19:15.746000128
4693,0025CB05,1.51,11.68,2021-04-16 11:19:15.788999936


That format seems more normal 
We just need to find the min and max value [min and max](https://www.kite.com/python/answers/how-to-find-the-max-value-of-a-pandas-dataframe-column-in-python)

```
minimum_of_a_column = dataframe.a_collumn.min()
```

In [7]:
min = df.time.min()
max = df.time.max()
duration = max-min
print(duration)

0 days 00:02:55.621999872


### 4.3.3.3 **TODO** individual duration

We know how many tags, and their individual ID's, and the complete duration for the dataframe

In the cell below, please implement a for-loop, that extract the duration for the individual tags. 

1. Create a for loop that loops over tag in unique tags 
2. Extract the minimum timestamp 
    - to get the data belonging to only tag use ```data_for_tag = df.loc[df.tag_id==tag]``` , where tag is an id in the unique_tags array
3. Add a line, that prints the information in each loop use the print function as follows ```print(f"text that needs to be printed {variable} remaining text")``` 
    - e.g., ```print(f"The data duration for tag {tag} is {duration}")```

The output could look similar to:

```
The data duration for tag 0025CB03 is 0 days 00:02:55.576000
The data duration for tag 0025C9EB is 0 days 00:02:54.980999936
The data duration for tag 0025C9E6 is 0 days 00:02:54.399000320
The data duration for tag 0025CAF6 is 0 days 00:02:54.656000256
The data duration for tag 0025CB05 is 0 days 00:02:45.862999808
```

In [8]:
### 4.3.3.3 **TODO** individual duration


### 4.3.3.4 **TODO** individual duration in percentage

We know how many tags, and their individual ID's, their individual duration.

Now we want to find their individual duration presented as percentage indexed with the maximum duration.

In the cell below, please implement a for-loop, that extract the duration for the individual tags, and appends the value to an List that holds all durations.
```
list = [1,2,3]
list.append(4)

# now the list contains [1,2,3,4]

``` 
### Task
1. Create/extend for loop, that calculates the percentage indexed with the maximum duration

The output could look similar to:

```
The data duration for tag 0025CB03 is 0 days 00:02:55.576000 ~ 100.0%
The data duration for tag 0025C9EB is 0 days 00:02:54.980999936 ~ 99.66%
The data duration for tag 0025C9E6 is 0 days 00:02:54.399000320 ~ 99.33%
The data duration for tag 0025CAF6 is 0 days 00:02:54.656000256 ~ 99.48%
The data duration for tag 0025CB05 is 0 days 00:02:45.862999808 ~ 94.47%
```

In [9]:
### 4.3.3.4 **TODO** individual duration in percentage


## 4.3.4 Plotting the trajectory data
In python there are several libraries for plotting data, and Matplitlib is one of those. Matplotlib offers a great amout of features (if you are used to matlab, matplotlib is rather intuitive)

[Matplotlib tutorial](https://matplotlib.org/stable/tutorials/introductory/pyplot.html)

The tutorial contains this example, which we will take inspiration from (but slightly change):

```
import matplotlib.pyplot as plt
names = ['group_a', 'group_b', 'group_c']
values = [1, 10, 100]
label = "data"

figure = plt.figure(figsize=(12, 4))

ax1 = figure.add_subplot(131)
ax1.bar(names, values, label)
ax2 = figure.add_subplot(132)
ax2.scatter(names, values)
ax3 = figure.add_subplot(133)
ax3.plot(names, values)
```


### 4.3.4.1 **TODO** Run plotting example
1. Paste the above code snippet into below cell and understand, what is happening
2. Add x- and y labels to the bar plot using ```ax.set_xlabel("label")``` N.B. ax is the plot object from ```ax=figure.add_subplot```
3. Add plot title to the bar plot using ```ax.set_title("title")```
4. Add legends the bar plot using ```ax.legend()```
5. Add a title for the figure ```figure.suptitle('figure title')```
6. Understand the numbers in ```figure.add_subplot(numbers)``` (Double click to edit/ comment on the points below)
    - First digit = ?
    - Second digit = ?
    - Third digit = ?

This should result in:

<img src="Images/4.3.4.1.png" style="padding-bottom:5px;" height="300px"/>


In [10]:
### 4.3.4.1 **TODO** Run plotting example


### 4.3.4.2 **TODO**  Plot the trajectory of all the tags
Please in the following cell do:
1. Create a figure with ```figure = plt.figure(figsize=(10,10))```
2. Add a subplot in the upper left corner( first position ) of a 2 rows, 2 columns figure ```ax = figure.add_subplot(position_as_described)```
3. Create for-loop, that iterate over the ids of the unique tags ```for tag in unique_tags:```
4. Isolate the data for the tag ```data_tag = df.loc[da.tag_id==tag]```
5. plot the trajctories on the plot object called ```ax``` Notice that the ```ax```-object is created outside of the loop, the plots are appended
    - ```ax.plot(data_tag.x, data_tag.y, '-', label=tag, linewidth=1)```
    - ```data_tag.x``` accesses the x values of data_tag, same for y
    - '-' is making a solid line, -- dashed, : dotted, .- dot_dash
    - Adding r in front would make a red solid line 'r-' etc. 
6. set x_label=X [m], y_label=Y [m], title = Original data, and activate legend (rember the qoutes(") to turn "X [m]" into a string)


Should result in a plot like this:

<img src="Images/4.3.4.2.png" style="padding-bottom:5px;" height="300px"/>


In [11]:
### 4.3.4.2 **TODO**  Plot the trajectory of all the tags



## 4.3.5 Moving on to the next step of handling data in this programming approach
The data that you have just plotted seems rather noisy. In fact almost all sensor data is noisy.... 

To handle noisy dat, we use filters. There exists (as you may know) wast amounts of filters, spanning from very simple to more sophisticated, e.g. moving average to Kalman respectivly. 
Filters are designed for different application, audio, images, and trajectory data. 

Lets try the simple moving average filter


## 4.3.5.1 **TODO** Application of moving average filter

1. Add a subplot to the figure, we created in previos code cell, on the seccond position
2. Isolate the data of the first tag ```data_tag = df.loc[df.tag_id==unique_tags[0]]```
3. Apply the build-in mooving, and average functionality from pandas
    - the moving function is called rolling in pandas and is used ```datafarame.column.roling(window).mean()```, where the window is the number of rows, tht is avaraged with the ```mean()```
    - Example ```data_tag.x = data_tag.x.rolling(10).mean()```
    - do the same for the y collumn
4. plot the filtered data on the subplot you have just added
5. Add plot information
    - Add title = Moving Avg. W=10
    - Add x- and y label
    - Add legends

N.B. Ignore the Warnings about the efficiency

Should result in a plot like this:

<img src="Images/4.3.5.1.png" style="padding-bottom:5px;" height="300px"/>

In [12]:
### 4.3.5.1 **TODO** Application of moving average filter



## 4.3.5.2 **TODO** Application of moving average filter Change the window size

1. Add a subplot to the figure, we created in previos code cell, on the third position
2. Isolate the data of the first tag ```data_tag = df.loc[df.tag_id==unique_tags[0]]```
3. Apply the build-in moving, and average functionality from pandas as before, but with a window size of **20**.
4. plot the filtered data on the subplot you have just added
5. Add plot information



N.B. Ignore the Warnings about the efficiency

Should result in a plot like this:

<img src="Images/4.3.5.2.png" style="padding-bottom:5px;" height="300px"/>

In [13]:
### 4.3.5.2 **TODO** Application of moving average filter Change the window size


## 4.3.5.3 **TODO** Application of moving average filter Change the window size

1. Add a subplot to the figure, we created in previos code cell, on the fourth position
2. Isolate the data of the first tag ```data_tag = df.loc[df.tag_id==unique_tags[0]]```
3. Apply the build-in moving, and average functionality from pandas as before, but with a window size of **30**.
4. plot the filtered data on the subplot you have just added
5. Add plot information



N.B. Ignore the Warnings about the efficiency

Should result in a plot like this:

<img src="Images/4.3.5.3.png" style="padding-bottom:5px;" height="300px"/>

In [14]:
### 4.3.5.3 **TODO** Application of moving average filter Change the window size

## 4.3.5.4 **TODO** Describe, what happens with the data, when we apply moving average
Please comment on the point below (Double click to edit/comment)
1. Does the filter exclusively impact the data positively?
    - ?
2. Does the data look like human trajectories?
    - ?
3. What window size looks most like what you expect from human trajectories
    - ?
4. How could we test if we are right?   
    - ?


## 4.3.6 Building our own domain knowledge filter
We could for example use some domain knowledge about where the tags were applied

What would that mean to the speeds that are represented in the data displacmen/delta time?

We refer to this type of filtering, domain knowledge based filtering, as we e.g., can derive maximum verlocities if we know who/what carried the tag.

An easy way to extract verlocity and displacement information from a trajectory is to use [get_derivatives()](https://traja.readthedocs.io/en/latest/calculations.html#derivatives), which is a part of the Traja package that we installed earlier.

### 4.3.6.1 Side note on "masking" data in pandas
How do we "mask" data ? 
A mask is a series of indexes and boolean values.
```
0. True
1. False
.. ....
30. False
31. etc....
```
In order to create such a mask we "ask" a question imagine this dataframe
```
dataframe = 

idx height  gender
0.  180     male
1.  200     female
2.  160     male
3.  170     non-binary
```
Then we could as for the entries, where gender is male, and height is more than 170cm 

```
mask = (dataframe.gender==male) & (dataframe.height>=170)

```
**N.B.** & is a boolean operator to do the logical conjunction also refered to as AND if unfamiliar find more information and truth table here  [Truth table for logical operators](https://www.wikiwand.com/en/Truth_table) 

resuling in: 
```
mask = 
0. True
1. False
2. False
3. False
```

Now we can use this mask to get the dataframe that fulfills our query:

```
queried_data = datafram.loc[mask]
queried_data = 

idx height  gender
0.  180     male
```
Another way to do this is to create the mask in the query

```
queried_data = datafram.loc[(dataframe.gender==male) & (dataframe.height>=170)]
queried_data = 

idx height  gender
0.  180     male
```
here is some further information on [Masks](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mask.html) in pandas 

### 4.3.6.2 Domain knowledge 

Now we know how to mask and query data lets figure out, what to query.

We know:
1. Only human workers did carrie the tags not machinery.
2. Humans do not walk faster than 7.2 km/h ~ 2 m/s as they should not run on a construction site 


#### 4.3.6.3 **TODO** Lets create our domain knowledge filter
1. Create a new figure with the size 10 by 10 to hold the following plots 
2. Add a plot in the first position of a 2 by 2 grid
3. Create a variable called *tag* to hold the first ID of unique ids
3. Isolate data from the first *tag*
4. Traja does unfortunately not understand the date_time format, which means we have to convert the time to nano seconds
    - Use the following line to do so ```data_tag.time = (data_tag.time.values.astype(float))*10**-9```
5. Plot the original/non-filtered data in the first plot
6. Add plot information
7. extract the derivatives for each row using ```derivatives = dataframe.traja.get_derivatives()```
8. Print the first lines / head() of the derivatives-dataframe 
    - Name the columns that exist in the dataframe
    - ?
9. Create a mask, that is True for the for the rows, where the speed is below 2 m/s
10. Create another mask, which is True for values above -2 m/s
11. Make a resulting mask, that is True for values above -2 m/s *AND* below 2 m/s
12. Queary this data into a variable called "speed_filtered_df"
13. Add a plot to the second position of the figure we made before
14. Plot the speed filtered data in th plot
15. Add plot information
16. Explain with words
    - What did that do to the data, if you compare to the output the original/unfiltered data in position one?
    - What if you compare to the output of our moving average filter in the other figure?

In [15]:
### 4.3.6.3 **TODO** Lets create our domain knowledge filter




### 4.3.6.4 **TODO** Smoothening the output in a more sophisticated way
We have used the moving average filter for smoothening, but we did this before we used our domain knowledge filter.

1. Lets plot the output of the moving average filter, we find most reasonable, but this time on the domain filtered data
    1. Add a sub plot to the third position
    2. Create a copy of the speed filtered data using ```copy_for_mean = speed_filtered.copy()```
    3. Apply the moving average filter to the speed filtered data 
    4. Plot the data
    5. Add plot information
2. Do the same for a [Savitzky–Golay](https://www.wikiwand.com/en/Savitzky%E2%80%93Golay_filter) filter.
    1. Add a sub plot in the fourth position of the figure
    2. Create a copy of the speed filtered data using ```copy_for_sg = speed_filtered.copy()```
    3. Apply the Savitzky–Golay filter to the speed filtered data (use window size of 7, and polynomial order of 3)
        - ```sg_filtered_df = traja.smooth_sg(dataframe_to_filter, w=window_size, p=polynomial_order)``` 
    4. plot the data
    5. Add plot information


#### Info on Savitzky–Golay
The second filtering algorithm is a more sofisticated one. It is called [Savitzky–Golay](https://www.wikiwand.com/en/Savitzky%E2%80%93Golay_filter). 
It can be compared to fitting the data to a higher degree polynomial.

[traja.smooth_sg(data, window, polynomial_order)](https://traja.readthedocs.io/en/latest/calculations.html)


The result when you have performed 4.3.6.1-4.3.6.4 should look somthing like this:

<img src="Images/4.3.6.4.png" style="padding-bottom:5px;" height="300px"/>

In [16]:
### 4.3.6.4 **TODO** Smoothening the output in a more sophisticated way



### 4.3.6.5 **TODO** Experimenting with the Savitzky–Golay filter

1. Create a figure to invesigate the Savitzky–Golay
2. Add four sub plot for:
    1. moving average with a window size of 9
    2. sg_smooth with a window size of 7, and polynomial order of 5
    3. sg_smooth with a window size of 9, and polynomial order of 3
    3. sg_smooth with a window size of 9, and polynomial order of 1
3. Describe what happens, and why the output makes sense.
    - what does the polynomial degree mean?


### 4.3.6.6 **TODO** Lets wrap our filter in a function for easy application
Here is some info on [creating functions](https://www.w3schools.com/python/python_functions.asp) but we also saw that in the basics file 03.02

1. Define the function ```def function_name(input_one, input_two, ....., input_N)```
    - The function should take the 
        - data_frame, 
        - window size, 
        - polynomial_degree
2. Create a copy of the incomming data_frame
3. Convert the time ```data_copy_in.time = (data_copy_in.time.values.astype(float))*10**-9```
4. Create a output_dataframe, whith the same collums as the incomming ```output_dataframe=pd.DataFrame(columns=data_copy_in.columns)```
5. Make a for loop that iterates over the tag of incomming data similar to  ```for tag in incomming_dataframe.tag_id.unique():``` and do following in the loop
    1. Isolate data for the tag in tag_dataframe
    2. Get derivatives
    3. Filter speeds below -2 m/s and above 2 m/s
    4. Perform Savitzky–Golay smothening
    5. Append filtered output to the output_dataframe ```output_dataframe = output_dataframe.append(tag_dataframe)```
6. Return the output_dataframe whan loop has ended


In [17]:
### 4.3.6.6 **TODO** Lets wrap our filter in a function for easy application



## 4.3.7 **TODO** In area analyis

Now that we have an approach to filter the data, we can start to do som analysis that may reveal information that is usefull for construction management. 

What if we wanted to investigate how much time a tag (person) spend in a specific area? e.g., if an area is off limit due to safety resriction, progress analysis etc.

Here we have at least two options:
1. To use the mask approach that we just did - This is fine as we want to check a square bounding box
2. Use a polygon, and check wether a point is inside or not - handy if we wanna check more advances shapes like circles, triangles etc.  

Lets just keep it simple and try out the first approach
1. Define the area, we can do this with a dictionary
    ```
    area = {
        "x_min":11,
        "y_min":11,
        "x_max":13,
        "y_max":13
    }
    ```
3. filter the data, using our filter function defined above ```filtered_data = filter_function(df,7,3) ``` Remember df was the original data we loaded in the start 
4. Isolate the first tag in variable called *tag_data_filtered*
5. Create a mask that is True for values between x_min and x_max ```mask_x  = ((tag_data_filtered.x >= area["x_min"]) & (tag_data_filtered.x<= area["x_max"]))``` notice how we index a dictionary
6. Create a similar mask for y
7. Create the resulting mask of mask_x *AND* mask_y 
8. Isolate data inside our area and call the variable *data_inside_area*
9. Understand and use the following code to analyze the visits:
    ```
    list_of_continues_visits=[]
    current_visit=[]
    for row in enumerate(data_inside_area.iterrows()):
        time_for_row =  row[1]["time"]
        if not current_visit: # check if empty
            current_visit.append(time_for_row)
            continue
        elif time_for_row-current_visit[-1]<1: # less than one seccond
            current_visit.append(time_for_row)
            continue
        elif time_for_row-current_visit[-1]>=1: # more than or equal to one seccond
            if len(current_visit)>1:
                list_of_continues_visits.append(current_visit)
            current_visit = [] #empty current visit, and continue

    for idx,visit in enumerate(list_of_continues_visits):
        duration  = visit[-1]-visit[1]
        print(f"Visit #{idx} had a duration of {round(duration,2)} s")
    ```


In [18]:
## 4.3.7 TODO In area analyis



## 4.3.8 **TODO** Proximity analysis 
There can be several reasons to be interested in a persons proximity to objects, heavy macinery such as crane hook, excavator , or event other persons (e.g., Covid-19 regualtion, or stike by accidents) 

1. Get a filtered dataset
2. Get the first available tag id 
3. Isolate the data for the tag 
4. Isolate the remaining data 
    - hint '~' will invert a mask
5. Create a empty dataframe with the identical columns as the input dataframe 
    - Use ```new_empty_dataframe_with_similar_columns = pd.DataFrame(columns=input_dataframe.columns)```
6. Iterate over the row of data found in pt 3. use:```for row in data_for_tag.iterrows():```
    - Iterate over the row of data found in pt 4.
        - check if the time of the two rows are within 1 second
        - if time is within; calculate the euclidian distance between the rows
        - if the distance is less than 1 meter append the row of the remaining data to the empty data frame from pt. 5 use:
            - ```new_empty_dataframe_with_similar_columns = new_empty_dataframe_with_similar_columns.append(row_2[1])```
7. Plot the results in a figure, with only one plot, and with informations

Result should be simsilar to: 

<img src="Images/4.3.8.png" style="padding-bottom:5px;" height="300px"/>

In [19]:
### 4.3.8 **TODO** proximmity analysis


## 4.3.9 **TODO** Incident report analysis
Can we say somthing more detailed about the incidents ?

Try to see how far you can get in the following tasks. You have all the tools available from the above assignments, but you will have to put the pieces together. 

Feel free to ask for help!

1. How many incidents were mad by each individual tag?
    - For one tag it may look like:
        ```
        tag_id 0025C9EB had 11 proximity incidents with 0025CB03
        tag_id 0025C9E6 had 4 proximity incidents with 0025CB03
        tag_id 0025CAF6 had 0 proximity incidents with 0025CB03
        tag_id 0025CB05 had 6 proximity incidents with 0025CB03
        ```
2. Turn te above into a function and run in for loop and make the analysis for all tags
3. Print a nice report using the print(f"text {variable}") way
    - Could look similar to :
        ```
        tag_id 0025CB03 had 21 incidents in total
            11 proximity incidents with 0025C9EB
            4 proximity incidents with 0025C9E6
            0 proximity incidents with 0025CAF6
            6 proximity incidents with 0025CB05
        tag_id 0025C9EB had 51 incidents in total
            11 proximity incidents with 0025CB03
            15 proximity incidents with 0025C9E6
            3 proximity incidents with 0025CAF6
            22 proximity incidents with 0025CB05
        tag_id 0025C9E6 had 30 incidents in total
            4 proximity incidents with 0025CB03
            15 proximity incidents with 0025C9EB
            3 proximity incidents with 0025CAF6
            8 proximity incidents with 0025CB05
        tag_id 0025CAF6 had 7 incidents in total
            0 proximity incidents with 0025CB03
            3 proximity incidents with 0025C9EB
            3 proximity incidents with 0025C9E6
            1 proximity incidents with 0025CB05
        tag_id 0025CB05 had 37 incidents in total
            6 proximity incidents with 0025CB03
            22 proximity incidents with 0025C9EB
            8 proximity incidents with 0025C9E6
            1 proximity incidents with 0025CAF6
        ```
4. Create a confusion matrix-plot for easier overview
    - Could look similar to:

        <img src="Images/4.3.9.4.png" style="padding-bottom:5px;" height="300px"/>

5. Calculate the number of incidents in [20cm, 40cm, 60cm, 80cm, 100cm] and make bar-plot 
    - Could look similar to:
        
        <img src="Images/4.3.9.5.png" style="padding-bottom:5px;" height="300px"/>

In [20]:
## 4.3.9.1 **TODO** Incident rwith other tags 


In [21]:
## 4.3.9.2 **TODO** Incident with other tags for all tags 

In [22]:
## 4.3.9.3 **TODO** Incident reporting

In [23]:
## 4.3.9.4 **TODO** Incident reporting in confusion matrix


In [24]:
## 4.3.9.5 **TODO** Interval analysis of incidents and barplot

